# Reviews sentiment analysis and metadata creation.

Here are the requirements that I used.

In [42]:
import os
import pandas as pd
import re
import pickle

In [8]:
import nltk
# nltk.download('punkt') # Uncomment this line to download the nltk punkt resource.

In [9]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
# https://www.nltk.org/api/nltk.stem.html
# https://tartarus.org/martin/PorterStemmer/
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [10]:
# https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/Gensim%20Newsgroup.ipynb
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

Change this for your path.

In [11]:
os.chdir('F:\Data\Films\ReviewsSentiment')
print(os.getcwd())

F:\Data\Films\ReviewsSentiment


In [12]:
sentiment_df = pd.read_csv('IMDB Dataset.csv')

In [13]:
print(sentiment_df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


The reviews data itself is not in a good format for getting anything done on it.
    I will implement tokenization, stemming, and lemmatization to clean the paragraphs.
    NLTK and SKLEARN stopwords sets are used for the stopwords removal.
    NLTK tokenizer, PorterStemmer, Lemmatizer

In [14]:
reviews_raw = sentiment_df['review'].tolist()
# print(reviews_raw)

In [9]:
print(reviews_raw[:1])

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [15]:
stop_words1 = set(stopwords.words('english'))
# print(stop_words1)
stop_words2 = set(ENGLISH_STOP_WORDS)
# print(stop_words2)
stop_words3 = set('it')

In [ ]:
print(stop_words1)
print(stop_words2)
print(stop_words3)

These both have their ups and downs
    splitting on space get's things like "you'll" where the nltk tokens will seperate on the " ' "
    splitting on space however will miss things like "word....banana" or grammatical mistakes.
    Both will be used, test the difference 

In [17]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def remove_stop_words(item, space=False, join=True):
    if space:
        # space split
        tokenized_item = item.split(' ')
    else:
        # Use of the nltk tokenizer
        tokenized_item = word_tokenize(item)
    tokenized_stopped_item = []
    for word, i in zip(tokenized_item, range(len(tokenized_item))):
        if word.lower() in stop_words1 or word.lower() in stop_words2 or word.lower() in stop_words3:
            pass
        else:
            tokenized_stopped_item.append(lemmatizer.lemmatize(stemmer.stem(word.lower())))
    if join:
        return " ".join(tokenized_stopped_item)
    return tokenized_stopped_item

In [18]:
def cleaning_films(reviews_raw):
    space_changed_list = []
    nltk_changed_list = []
    for item, i in zip(reviews_raw, range(len(reviews_raw))):
        item_removed = re.sub('<br />', '', item)
        space_changed_list.append(item_removed)
        nltk_changed_list.append(item_removed)
        space_changed_list[i] = remove_stop_words(space_changed_list[i], space=True)
        nltk_changed_list[i] = remove_stop_words(nltk_changed_list[i], space=False)
    return space_changed_list, nltk_changed_list

In [19]:
cleaned_reviews_space, cleaned_reviews_nltk = cleaning_films(reviews_raw)

In [ ]:
pickle.dump(cleaned_reviews_space, open('cleaned_reviews_space.pickle', 'wb'))
pickle.dump(cleaned_reviews_nltk, open('cleaned_reviews_nltk.pickle', 'wb'))

In [57]:
cleaned_reviews_space = pickle.load(open('cleaned_reviews_space.pickle', 'rb'))
cleaned_reviews_nltk = pickle.load(open('cleaned_reviews_nltk.pickle', 'rb'))

In [ ]:
print(cleaned_reviews_space[:1])

In [ ]:
print(cleaned_reviews_nltk[:1])

The nltk set:

In [21]:
# Stop words are being removed before this
# 2-grams perhaps
count_vectorizer_nltk = CountVectorizer(ngram_range=(1, 3), max_df=.91, min_df=.005)
tfidf_transformer_nltk = TfidfTransformer()

In [22]:
counts_vectors_nltk = count_vectorizer_nltk.fit_transform(cleaned_reviews_nltk)
counts_features_nltk = count_vectorizer_nltk.get_feature_names()

This is nice to have as a pandas dataframe for one of the functions i had created a few weeks ago for this kind of data.

In [23]:
reviews_count_df = pd.DataFrame(counts_vectors_nltk.todense(), columns=counts_features_nltk)
class_labels = sentiment_df['sentiment'].tolist()

These should give (hopefully) a good classifier once it's in tf-idf vectors.

For now tho we can make some meta data off the counts of each of our feature words.
    Just a note the next function takes a bit to run on the settings from above, for simplification we can run this one time and save the dataframe that we used.

In [ ]:
# This function was created 2/17/2020 in android malware dectection, some edits
# names have been changed for more clear reference to this materiial
review_dict = dict()
feature_list = list()
for col_name, col_data in reviews_count_df.loc[:, 0:reviews_count_df.shape[1]].iteritems():
    feature_list.append(col_name)
    positive = 0
    negative = 0
    num = 0
    for i in range(len(col_data)):
        if col_data[i] == 1:
            num += 1
            if class_labels[i] == 'positive':
                positive += 1
            else:
                negative += 1
    if negative == 0:
        positive_percent = 1
    else:
        positive_percent = float(positive / num)
    negative_percent = 1 - positive_percent
    df = num / len(class_labels)
    review_dict[col_name] = [positive, negative, positive_percent, negative_percent, num, df]

Transformation of the count matrix into the tfidf matrix

In [ ]:
tfidf_transformer = TfidfTransformer()
reviews_tfidf = tfidf_transformer.fit_transform(counts_vectors_nltk)

In [53]:
print(tfidf_transformer.idf_)
print(type(tfidf_transformer.idf_))

[5.70170109 3.00814604 5.18779651 ... 5.44307134 5.22128926 6.25527788]
<class 'numpy.ndarray'>


In [19]:
meta_data = pd.DataFrame(review_dict).T
meta_data['idf'] = tfidf_transformer.idf_

In [20]:
meta_data.columns = ['positive', 'negative', 'positive_percent', 'negative_percent', 'total_calls', 'term_frequency', 'idf']

In [29]:
meta_data = pd.read_csv('reviews_col_meta.csv')

In [30]:
meta_data_3 = meta_data.loc[meta_data['idf'] > 3.3]
meta_data_4 = meta_data.loc[meta_data['idf'] > 4]

In [31]:
meta_data.to_csv('reviews_col_meta.csv')
meta_data_3.to_csv('reviews_col_meta_3.csv')
meta_data_4.to_csv('reviews_col_meta_4.csv')

You should save this to github.

This is not the end of the feature selection process.

Go to Tab document for some justifying graphs.

In the tab file we see that above a 3.3 is a good idf that we can use to get some classifications, 4 was also created to see what could come out of it.  

I am going to create a list with the three vocabularies in a list so that we can use each of them to create models.
Just a note I am going to make a save for the vocabularies to save on some space.

In [ ]:
vocabularies = [meta_data['name'], meta_data_3['name'], meta_data_4['name']]

In [ ]:
pickle.dump(vocabularies, open('vocabularies.pickle', 'wb+'))

In [ ]:
print(os.getcwd())
vocabularies = pickle.load(open('vocabularies.pickle', 'rb'))

In [67]:
for item in vocabularies:
    print(item.shape)

(3073,)
(2972,)
(2759,)


Now that we have some of the corpuses created we can create some topic models that use the corpus.
    Don't use pyLDAvis SKlearn.  it doesn't work.
    I will use gensim.

In [ ]:
print(lda_tfidf.components_)